In [39]:
%store -r df y_target

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as plx

from sklearn.model_selection import train_test_split


In [41]:
# Define X as one hot encoding

X = pd.get_dummies(df, drop_first=True)

In [42]:
# Prepare y to be good predictable

y_values = y_target["loan_status"].value_counts()
y_fully_paid, fully_paid_val, fully_paid_str = list(y_values.index[[0,2,3,6,7]]), 0, "Fully Paid"
#y_delay, delay_val = list(y_values.index[[5,6,7]]), "charged_off"
y_charged_off, charged_off_val, charged_off_str = list(y_values.index[[1,4,5,8]]), 1, "Charged Off"

y_target["loan_status_int"] = y_target["loan_status"]
y_target["loan_status_int"] = y_target["loan_status_int"].apply(lambda x : fully_paid_val if x in y_fully_paid else x)
#y_target["loan_status_int"] = y_target["loan_status_int"].apply(lambda x : delay_val if x in y_delay else x)
y_target["loan_status_int"] = y_target["loan_status_int"].apply(lambda x : charged_off_val if x in y_charged_off else x)
y_target["loan_status_str"] = y_target["loan_status_int"].apply(lambda x : fully_paid_str if x == fully_paid_val else charged_off_str)

y_values = y_target["grade"].value_counts()

y_grade_AB, grade_AB_val = list(y_values.index[[0,1]]), 0
y_grade_CDEFG, grade_CDEFG_val = list(y_values.index[[2,3,4,5,6]]), 1

y_target["grade_int"] = y_target["grade"]
y_target["grade_int"] = y_target["grade_int"].apply(lambda x : grade_AB_val if x in y_grade_AB else x)
y_target["grade_int"] = y_target["grade_int"].apply(lambda x : grade_CDEFG_val if x in y_grade_CDEFG else x)

y_int_10, int_10_val = [5,6,7,8,9,10], 10
y_int_15, int_15_val = [11,12,13,14,15], 15
y_int_20, int_20_val = [16,17,18,19,20], 20
y_int_25, int_25_val = [21,22,23,24,25], 25


y_target["int_rate_int"] = df["int_rate"].astype("int")
y_target["int_rate_int"] = y_target["int_rate_int"].apply(lambda x : int_10_val if x in y_int_10 else x)
y_target["int_rate_int"] = y_target["int_rate_int"].apply(lambda x : int_15_val if x in y_int_15 else x)
y_target["int_rate_int"] = y_target["int_rate_int"].apply(lambda x : int_20_val if x in y_int_20 else x)
y_target["int_rate_int"] = y_target["int_rate_int"].apply(lambda x : int_25_val if x in y_int_25 else x)

0        0
1        1
2        0
3        0
4        0
        ..
42531    0
42532    0
42533    0
42534    0
42535    0
Name: loan_status_int, Length: 42535, dtype: int64

In [43]:
#Split into train and test

y_stratify = y_target["loan_status_int"].astype(str) + y_target["grade"]

X_train, X_test, y_stratify_train, y_stratify_test = train_test_split(X, y_stratify, test_size = 0.25, random_state = 70, stratify= y_stratify)

# Show the results of the split
print ("Training set has {} samples.".format(X_train.shape[0]))
print ("Testing set has {} samples.".format(X_test.shape[0]))
print()
print (f"Training set has values:\n {format(y_stratify_train.value_counts())}")
print (f"Testing set has values:\n {format(y_stratify_test.value_counts())}")

Training set has 31901 samples.
Testing set has 10634 samples.

Training set has values:
 0B    8153
0A    7180
0C    5428
0D    3518
0E    1886
1B    1138
1C    1126
1D     994
0F     665
1E     659
1A     458
1F     311
0G     254
1G     131
dtype: int64
Testing set has values:
 0B    2718
0A    2393
0C    1810
0D    1173
0E     629
1B     380
1C     376
1D     331
0F     222
1E     220
1A     152
1F     103
0G      84
1G      43
dtype: int64


In [44]:
y_stratify

0        0B
1        1C
2        0C
3        0C
4        0B
         ..
42531    0C
42532    0B
42533    0B
42534    0A
42535    0A
Length: 42535, dtype: object

In [45]:
# hier werden fehlende daten durch wahrscheinliche daten ersetzt (mean,median,blaa)
# Attention: X_train und X_test sepparat behandeln

X_train["open_acc"].fillna(X_train["open_acc"].median(),inplace=True)
X_train["annual_inc"].fillna(X_train["annual_inc"].median().round(2),inplace=True)

X_test["open_acc"].fillna(X_test["open_acc"].median(),inplace=True)
X_test["annual_inc"].fillna(X_test["annual_inc"].median().round(2),inplace=True)


/Users/kli/neuefische/14_Second_Project/NeueFische-2nd_Project/.venv/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/Users/kli/neuefische/14_Second_Project/NeueFische-2nd_Project/.venv/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/Users/kli/neuefische/14_Second_Project/NeueFische-2nd_Project/.venv/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in th

In [46]:
#### skalieren
from sklearn.preprocessing import MinMaxScaler

# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler()

numerical = [col for col in X if X.dtypes[col] == "float64"]
X_train.loc[:,numerical] = scaler.fit_transform(X_train[numerical])
X_test.loc[:,numerical] = scaler.transform(X_test[numerical])

/Users/kli/neuefische/14_Second_Project/NeueFische-2nd_Project/.venv/lib/python3.8/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/Users/kli/neuefische/14_Second_Project/NeueFische-2nd_Project/.venv/lib/python3.8/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


In [47]:
# Handle imbalanced classes
from imblearn.over_sampling import SMOTE

smote = SMOTE()

# fit predictor and target variable
X_train, y_stratify_train = smote.fit_resample(X_train, y_stratify_train)


In [48]:
y_loan_status_train = y_stratify_train.apply(lambda x: int(x[0]))
y_grade_train = y_stratify_train.apply(lambda x: x[1])

y_loan_status_test = y_stratify_test.apply(lambda x: int(x[0]))
y_grade_test = y_stratify_test.apply(lambda x: x[1])

y_loan_status_str_train = y_loan_status_train.apply(lambda x : fully_paid_str if x == fully_paid_val else charged_off_str)
y_loan_status_str_test = y_loan_status_test.apply(lambda x : fully_paid_str if x == fully_paid_val else charged_off_str)

In [49]:
%store X_train X_test y_loan_status_train y_loan_status_test y_loan_status_str_train y_loan_status_str_test y_grade_train y_grade_test


Stored 'X_train' (DataFrame)
Stored 'X_test' (DataFrame)
Stored 'y_loan_status_train' (Series)
Stored 'y_loan_status_test' (Series)
Stored 'y_loan_status_str_train' (Series)
Stored 'y_loan_status_str_test' (Series)
Stored 'y_grade_train' (Series)
Stored 'y_grade_test' (Series)
